# Hiru gorputzeko problemaren soluzio periodikoen bilaketa, fasez fase

<ul id="top">
<li><a href="#Pakete-eta-funtzioak-kargatu"> 
    Pakete eta funtzioak kargatu</a></li>
<li><a href="#1.-Fasea:-Ausazko-hasierako-egoera-lortu">
    1. Fasea: Ausazko hasierako egoera lortu</a></li>
<li><a href="#2.-Fasea:-$z$-taulako-$z_3$-eta-$z_4$-osagaiak-optimizatu">
    2. Fasea: $z$ taulako $z_3$ eta $z_4$ osagaiak optimizatu</a></li>
<li><a href="#3.-Fasea:-$z$-taula-osoa-optimizatu">
    3. Fasea: $z$ taula osoa optimizatu</a></li>
<li><a href="#4.-Fasea:-soluzio-periodikoaren-periodoa,-$T$-optimizatu-$z$-taularekin-batera">
    4. Fasea: soluzio periodikoaren periodoa, $T$ optimizatu $z$ taularekin batera</a></li>
<li><a href="#5.-Fasea:-Sistemaren-energia,-$h$,-moldatu-$T$-eta-$z$-taularekin-batera">
    5. Fasea: Sistemaren energia, $h$, moldatu $T$ eta $z$ taularekin batera</a></li>
<li><a href="#Une-kolinealak-aztertu">
    Une kolinealak aztertu</a></li>
<li><a href="#Fast-Fourier-Transform">
    Fast Fourier Transform</a></li>
<li><a href="#Minimo-lokal-batean-al-gaude?">
    Minimo lokal batean al gaude?</a></li>   
</ul>  

## Pakete eta funtzioak kargatu

In [ ]:
using LinearAlgebra
using Plots 
using Optim
using BenchmarkTools
using Combinatorics
using CSV
using DataFrames
using ForwardDiff
using FFTW

In [ ]:
include("./src/EMI.jl"); include("./src/TBP.jl"); include("./src/utils.jl"); include("./src/Loss.jl");

<a href="#top">Hasierara</a>
## 1. Fasea: Ausazko hasierako egoera lortu

In [ ]:
z0 = pi*rand(4)
u0 = HasierakoEgoera(z0)
U0 = LCinvFcn(u0)
h = ThreeBodyRelEnergy(u0)
kont = 1
while (E_bitarra(U0, rel2abs(u0))>0 || TBP_Bitarra(U0, 1/256, 50., h))
    z0 = pi*rand(4)
    u0 = HasierakoEgoera(z0)
    U0 = LCinvFcn(u0)
    kont+=1
    println(kont)
end

In [ ]:
@time u0 = HasierakoEgoera(z0)
U0 = LCinvFcn(u0)
println([ThreeBodyHamiltonianLC(U0), -ThreeBodyRelPotential(u0), ThreeBodyRelEnergy(u0)])
println([AngularMomentum(rel2abs(u0)),g(U0)])

### Parametroak definitu

In [ ]:
tauend = 20.
h = ThreeBodyRelEnergy(u0)
par = h
dtau = 1/256

### Erdiguneko metodo inplizituaren soluzioaren irudikapena
Erdiguneko metodo inplizitua erabiliz, dtau luzerako urrats bakoitzean problemaren egoera bildu eta grafiko batean erakusten du.

In [ ]:
tt, UU, uu = Irudikatu(z0, 0., tauend, dtau, par)
IrudikatuIELog(tt, UU,uu)

### Energia errore eta errore hamiltondarrak

In [ ]:
yrange = (-20, 0)
energia_erroreakTP = [ThreeBodyRelEnergy(u)/h - 1  for u in uu]
energia_errore_lokalakTP = energia_erroreakTP[2:end]-energia_erroreakTP[1:end-1]

pl1 = plot(tt, log10.(abs.(energia_erroreakTP)), legend = false,  title = "Energia errorea TwicePrecision", ylims=yrange)

In [ ]:
yrange = (-15, 0)
energia_erroreakTP = [ThreeBodyRelEnergy(u)/h - 1  for u in uu]
energia_errore_lokalakTP = energia_erroreakTP[2:end]-energia_erroreakTP[1:end-1]

pl1 = plot(tt, log10.(abs.(energia_erroreakTP)), legend = false,  title = "Energia errorea TwicePrecision", ylims=yrange)

In [ ]:
IrudikatuErrEH(tt, UU, uu)
IrudikatuRet(tt, UU)

<a href="#top">Hasierara</a>
## 2. Fasea: $z$ taulako $z_3$ eta $z_4$ osagaiak optimizatu

In [ ]:
z = copy(z0)
F0 = FLoss0(z,dtau,tauend,h)
w0 = [z[3], z[4]]
helb_minn,ss = F0(w0,3)
@time T, sigma = F0(w0, 2)
(helb_minn, T, sigma)

In [ ]:
@time res0 = optimize(F0,w0)

In [ ]:
w1 = Normalizatu_z(res0.minimizer)
helb_minn, ss = F0(w1, 3);
T, sigma = F0(w1, 2)

(helb_minn, T, sigma)

In [ ]:
pl = plot([0:T/F0.dtau], ss[1:Int64(T/F0.dtau)+1], title="title", legend = false)

In [ ]:
z1 = copy(F0.z)
z1[3:4] .= w1
tt,UU,uu = Irudikatu(z1, 0., T, F0.dtau, F0.h);

<a href="#top">Hasierara</a>
## 3. Fasea: $z$ taula osoa optimizatu

In [ ]:
F1 = FLoss1(dtau,tauend,h)
helb_minn,ss = F1(z1,3)
@time T, sigma = F1(z1, 2)
(helb_minn, T, sigma)

In [ ]:
@time res1 = optimize(F1,z1)

In [ ]:
z1_opt = Normalizatu_z(res1.minimizer)
helb_minn,ss = F1(z1_opt,3)
@time T, sigma = F1(z1_opt, 2)
(helb_minn, T, sigma)

In [ ]:
tt, UU, uu = Irudikatu(z1_opt, 0., T, F0.dtau, F0.h);

<a href="#top">Hasierara</a>
## 4. Fasea: soluzio periodikoaren periodoa, $T$ optimizatu $z$ taularekin batera

In [ ]:
Z0 = copy(z1_opt)
push!(Z0, T)
F2 = FLoss2(sigma,Int64(ceil(Z0[5]/F1.dtau)),F1.h)
@time F2(Z0)

In [ ]:
@time res2 = optimize(F2, Z0, g_tol=1e-13, autodiff=:forward, method=BFGS())

In [ ]:
Zopt2 = Normalizatu_z(res2.minimizer)
tt, UU,uu = Irudikatu(Zopt2,0.,Zopt2[5], Zopt2[5]/F2.nsteps, F1.h);

<a href="#top">Hasierara</a>
## 5. Fasea: Sistemaren energia, $h$, moldatu $T$ eta $z$ taularekin batera

In [ ]:
W0= copy(Zopt2)
push!(W0, -0.5)
#W0= copy(z1_opt)
#push!(W0, T)
#push!(W0, -0.5)
F3 = FLoss3(sigma,Int64(ceil(W0[5]/F1.dtau)))
@time F3(W0)

In [ ]:
@time res3 = optimize(F3, W0, g_tol=1e-13, autodiff=:forward, method=BFGS())

In [ ]:
Zopt3 = Normalizatu_z(res3.minimizer)
tt, UU, uu = Irudikatu(Zopt3,0.,Zopt3[5], Zopt3[5]/F2.nsteps, Zopt3[6]);
g_col_list = [g_col(u) for u in uu]
code_middle, code_far, ind_kolinear_list = getCode(UU, g_col_list)
println("Kodea_far: ", code_far)
println("Kodea_middle: ", code_middle)

<a href="#top">Hasierara</a>
## Une kolinealak aztertu

In [ ]:
i = 0;

In [ ]:
i+=1
Irudikatu(Zopt3,0.,tt[ind_kolinear_list[i]], Zopt3[5]/F2.nsteps, Zopt3[6]); 

<a href="#top">Hasierara</a>
## Fast Fourier Transform

In [ ]:
II = I_fcn.(UU[1:end-1])
hII = rfft(II)/length(II)
loghII = log10.(abs.(hII))
scatter(loghII)

<a href="#top">Hasierara</a>
## Minimo lokal batean al gaude?

In [ ]:
helb = F3(BigFloat.(Zopt3))

In [ ]:
gradF3 = ForwardDiff.gradient(F3,BigFloat.(Zopt3))

In [ ]:
hess = ForwardDiff.hessian(F3,Zopt3)

eigen(hess).values

In [ ]:
Z3 = Zopt3 - hess\gradF3

In [ ]:
Float64.(ForwardDiff.gradient(F3,BigFloat.(Z3)))

In [ ]:
Z3 = Float64.(Z3)

In [ ]:
(Float64.(F3(BigFloat.(Z3))), Float64.(F3(BigFloat.(Zopt3))))

In [ ]:
tt, UU = Irudikatu(Zopt3,0.,Zopt3[5], Zopt3[5]/F3.nsteps, Zopt3[6]);

In [ ]:
IrudikatuIDeribatuak(tt, UU, LCFcn.(UU))